In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/modules')
import signateConnect

!pip install signate

signateConnect.connect()

Mounted at /content/drive
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=2981b81beb9d770a650882ab9ca6a4e540ca089e932f1a91997bc0bf1830bdc0
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Download 100%.


In [ ]:
! ls

drive  sample_data


In [ ]:
! signate list

  competitionId  title                                                                             closing     prize        submitters
---------------  --------------------------------------------------------------------------------  ----------  ---------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                              -                              6855
             24  【練習問題】お弁当の需要予測                                                      -                              8319
             27  【練習問題】Jリーグの観客動員数予測                                               -                              2032
            100  【練習問題】手書き文字認識                                                        -           Knowledge           331
            102  【練習問題】タイタニックの生存予測                                                -           Knowledge          2386
            103  【練習問題】音楽ラベリング                                                        -           Knowledge           156
            

In [2]:
! signate files --competition-id=983

  fileId  name        title                 size  updated_at
--------  ----------  ----------------  --------  -------------------
    1115  train.csv   学習用データ      28618194  2020-07-31 03:31:04
    1116  test.csv    テスト用データ     2917307  2020-07-31 03:31:32
    1270  submit.csv  サンプルファイル    242100  2020-10-01 19:14:52


In [3]:
! signate download --competition-id=983

submit.csv

test.csv

train.csv


Download completed.


In [57]:
! signate submit --competition-id=983 'submit.csv'

You have successfully submitted your predictions.We will send you the submission result to your email address.


In [4]:
import pandas as pd
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
sample_submit = pd.read_csv("submit.csv", index_col=0, header=None)

In [60]:
train.head()

,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
id,,,,,,,,,
0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid
1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff
2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid
3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid
4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid


In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242150 entries, 0 to 242149
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   loan_amnt          242150 non-null  float64
 1   term               242150 non-null  object 
 2   interest_rate      242150 non-null  float64
 3   grade              242150 non-null  object 
 4   employment_length  242150 non-null  object 
 5   purpose            242150 non-null  object 
 6   credit_score       242150 non-null  float64
 7   application_type   242150 non-null  object 
 8   loan_status        242150 non-null  object 
dtypes: float64(3), object(6)
memory usage: 18.5+ MB


In [5]:
data = pd.concat([train, test])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269050 entries, 0 to 269049
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   loan_amnt          269050 non-null  float64
 1   term               269050 non-null  object 
 2   interest_rate      269050 non-null  float64
 3   grade              269050 non-null  object 
 4   employment_length  269050 non-null  object 
 5   purpose            269050 non-null  object 
 6   credit_score       269050 non-null  float64
 7   application_type   269050 non-null  object 
 8   loan_status        242150 non-null  object 
dtypes: float64(3), object(6)
memory usage: 20.5+ MB


In [7]:
from sklearn.preprocessing import LabelEncoder

# ラベルエンコーディングを適用するための関数
def label_encode(column):
    le = LabelEncoder()
    return le.fit_transform(column)

In [8]:
data['term'] = data['term'].str.extract('(\d+)').astype(int)
data['grade'] = label_encode(data['grade'])
data['employment_length'] = data['employment_length'].str.extract('(\d+)').astype(float)
data['loan_status'] = data['loan_status'].map({'FullyPaid': 0, 'ChargedOff': 1})
data = pd.get_dummies(data, columns=['purpose', 'application_type'], drop_first=True)

In [9]:
# Z score正規化
from sklearn.preprocessing import StandardScaler
scaling_col = ['loan_amnt', 'interest_rate', 'credit_score']
scaler = StandardScaler()

data[scaling_col] = scaler.fit_transform(data[scaling_col])

In [10]:
train = data.loc[train.index]
test = data.loc[test.index]

In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242150 entries, 0 to 242149
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   242150 non-null  float64
 1   term                        242150 non-null  int64  
 2   interest_rate               242150 non-null  float64
 3   grade                       242150 non-null  int64  
 4   employment_length           242150 non-null  float64
 5   credit_score                242150 non-null  float64
 6   loan_status                 242150 non-null  float64
 7   purpose_credit_card         242150 non-null  uint8  
 8   purpose_debt_consolidation  242150 non-null  uint8  
 9   purpose_home_improvement    242150 non-null  uint8  
 10  purpose_house               242150 non-null  uint8  
 11  purpose_major_purchase      242150 non-null  uint8  
 12  purpose_medical             242150 non-null  uint8  
 13  purpose_moving

In [68]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26900 entries, 242150 to 269049
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   26900 non-null  float64
 1   term                        26900 non-null  int64  
 2   interest_rate               26900 non-null  float64
 3   grade                       26900 non-null  int64  
 4   employment_length           26900 non-null  float64
 5   credit_score                26900 non-null  float64
 6   loan_status                 0 non-null      float64
 7   purpose_credit_card         26900 non-null  uint8  
 8   purpose_debt_consolidation  26900 non-null  uint8  
 9   purpose_home_improvement    26900 non-null  uint8  
 10  purpose_house               26900 non-null  uint8  
 11  purpose_major_purchase      26900 non-null  uint8  
 12  purpose_medical             26900 non-null  uint8  
 13  purpose_moving           

In [11]:
test = test.drop(['loan_status'], axis=1)

In [12]:
target = train['loan_status']
train = train.drop(['loan_status'], axis=1)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

In [14]:
from imblearn.under_sampling import RandomUnderSampler
# ダウンサンプリング
sampler = RandomUnderSampler(random_state=42)
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [15]:
# データの重みづけを行います．
weights = target.value_counts()
class_weights = {0: weights[1] / weights[0], 1: 1.0}

In [29]:
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# 交差検証の設定
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# ハイパーパラメータの探索範囲
lgbm_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [15, 20, 25, 31, 50],
}

xg_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6, 7, 8],
}

# ハイパーパラメータの設定を行います.
lgbm_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'random_state': 42,
    'n_estimators': 100,
}

xg_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'n_estimators': 100,
}

# GridSearchCVを使ってハイパーパラメータチューニング
lgb = GridSearchCV(lgb.LGBMClassifier(**lgbm_params), lgbm_param_grid, cv=kf, scoring='f1', n_jobs=-1)
xg = GridSearchCV(xgb.XGBClassifier(**xg_params), xg_param_grid, cv=kf, scoring='f1', n_jobs=-1)

# # モデルの学習をします.
# lgb = lgb.fit(X_train, y_train)
# xg = xg.fit(X_train, y_train)

lgb_scores = []
xg_scores = []

for train_idx, val_idx in kf.split(X_train, y_train):
    # データの分割
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # lgbモデルの訓練と評価
    lgb_model = lgb.fit(X_train_fold, y_train_fold)
    lgb_scores.append(lgb_model.best_score_)

    # xgモデルの訓練と評価
    xg_model = xg.fit(X_train_fold, y_train_fold)
    xg_scores.append(xg_model.best_score_)

# 平均スコアを計算
lgb_avg_score = np.mean(lgb_scores)
xg_avg_score = np.mean(xg_scores)

print(f"Average LGBM score: {lgb_avg_score}")
print(f"Average XGB score: {xg_avg_score}")

# アンサンブル学習を行います.
ensemble = VotingClassifier(estimators=[('lgbm', lgb), ('xg', xg)], voting='soft')
ensemble.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 27061, number of negative: 27060
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 821
[LightGBM] [Info] Number of data points in the train set: 54121, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500009 -> initscore=0.000037
[LightGBM] [Info] Start training from score 0.000037
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 27060, number of negative: 27061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002450 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_

VotingClassifier(estimators=[('lgbm',
                              GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                                           estimator=LGBMClassifier(metric='binary_logloss',
                                                                    objective='binary',
                                                                    random_state=42),
                                           n_jobs=-1,
                                           param_grid={'learning_rate': [0.01,
                                                                         0.05,
                                                                         0.1],
                                                       'num_leaves': [15, 20,
                                                                      25, 31,
                                                                      50]},
                                           scoring='f1')),
                             ('xg',
                              GridSearchCV(cv=StratifiedKFold(n_splits=5, random...
                                                                   max_cat_threshold=None,
                                                                   max_cat_to_onehot=None,
                                                                   max_delta_step=None,
                                                                   max_depth=None,
                                                                   max_leaves=None,
                                                                   min_child_weight=None,
                                                                   missing=nan,
                                                                   monotone_constraints=None,
                                                                   multi_strategy=None,
                                                                   n_estimators=100,
                                                                   n_jobs=None,
                                                                   num_parallel_tree=None,
                                                                   random_state=42, ...),
                                           n_jobs=-1,
                                           param_grid={'learning_rate': [0.01,
                                                                         0.05,
                                                                         0.1],
                                                       'max_depth': [3, 4, 5, 6,
                                                                     7, 8]},
                                           scoring='f1'))],
                 voting='soft')

In [31]:
y_pred = (ensemble.predict_proba(X_test)[:, 1] >= threshold).astype(float)
y_pred

array([0., 0., 1., ..., 0., 1., 0.])

In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import numpy as np
# 混同行列の作成
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

# 混同行列をデータフレーム化
df_cm = pd.DataFrame(np.rot90(cm, 2), index=["実際のChargedOff", "実際のFullyPaid"], columns=["ChargedOffの予測", "FullyPaidの予測"])
print(df_cm)

# F1 Scoreの算出
f1 = f1_score(y_test, y_pred)
print(f1)

               ChargedOffの予測  FullyPaidの予測
実際のChargedOff           6009          2521
実際のFullyPaid           16256         23644
0.3902581587920117


In [30]:
from sklearn.metrics import accuracy_score
threshold = 0.5
lgb_preds = (lgb.predict_proba(X_test)[:, 1] >= threshold).astype(int)
xg_preds = (xg.predict_proba(X_test)[:, 1] >= threshold).astype(int)
ensemble_preds = (ensemble.predict_proba(X_test)[:, 1] >= threshold).astype(int)
lgb_acc = accuracy_score(y_test, lgb_preds)
xg_acc = accuracy_score(y_test, xg_preds)
ensemble_acc = accuracy_score(y_test, ensemble_preds)
print("ens", ensemble_acc)
print("lgb", lgb_acc)
print("xgboost", xg_acc)

ens 0.6122857732810242
lgb 0.6135659715052654
xgboost 0.6156308073508157


In [ ]:
from sklearn.metrics import f1_score, roc_curve

# 真のラベルと予測確率を用いてROC曲線を計算
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# 各閾値でのF1スコアを計算
f1_scores = [f1_score(y_test, [1 if p >= thr else 0 for p in y_pred]) for thr in thresholds]

# 最大のF1スコアとそのときの閾値を取得
max_f1_score = max(f1_scores)
optimal_threshold = thresholds[f1_scores.index(max_f1_score)]

print(f"Optimal Threshold based on F1 Score: {optimal_threshold:.4f}")


KeyboardInterrupt: ignored

In [35]:
y_pred.max()

1.0

In [54]:
pred = (ensemble.predict_proba(test)[:, 1] >= threshold).astype(int)

In [ ]:
print(min(pred))
print(len(pred))
print(sum(pred))
print(sum(pred) / len(pred))
print(max(pred))

0
26900
549
0.0204089219330855
1


In [ ]:
pred = [1 if p > 0.5 else 0 for p in pred]

In [ ]:
import numpy as np

def sigmoid(x):
  return 1/ (1 + np.exp(-x))

pred = sigmoid(pred)
pred = [1 if i > 0.5 else 0 for i in pred]

In [ ]:
import numpy as np

# 最小値と最大値を使用して0から1の範囲にスケーリング
pred = (pred - pred.min()) / (pred.max() - pred.min())

# 0.5を閾値として二値化
pred = np.where(pred >= 0.5, 1, 0)

In [55]:
sum(pred)

12024

In [56]:
sample_submit[1] = pred
sample_submit.to_csv('submit.csv', header=None, sep=',')

In [ ]:
%mv submit.csv ./drive/MyDrive/

In [ ]:
! ls

drive  sample_data  submit.csv	test.csv  train.csv
